## Chapter 2: Tiền xử lí số liệu (Preprocessing Data) 

Trong chương này chúng ta sẽ thực hành một số thao tác tiền xử lí số liệu đơn giản cho Machine Learning. 

### 2.1 Chuyển hóa biến có bản chất định tính về định lượng

In [79]:
# Gọi gói và hàm cần thiết
import numpy as np
from numpy import array

# Tạo một biến định  tính mô tả thông tin về giới tính: 
cat_data = array(["male", "female", "male", "male", "female", "male", "female", "female"])

# Xem qua: 
cat_data

array(['male', 'female', 'male', 'male', 'female', 'male', 'female',
       'female'],
      dtype='<U6')

In [80]:
# Viết một hàm chuyển hóa biểu thị rằng 1 ứng với female và ngược lại: 
from numpy import unique

def convert_to_num(data):
    # Liệt kê nhãn của biến định tính: 
    categories = unique(data)
    # Tạo một array trống: 
    features = []
    # Viết một vòng lặp for loop tạo biến nhị phân 0 - 1: 
    for cat in categories:
        binary = (data == cat)
        features.append(binary.astype("int"))
    return features

In [81]:
# Test hàm vừa viết cho cat_data: 
convert_to_num(cat_data)

[array([0, 1, 0, 0, 1, 0, 1, 1]), array([1, 0, 1, 1, 0, 1, 0, 0])]

### 2.2 Chuẩn hóa scale 0 - 1


In [82]:
# Viết hàm chuẩn hóa scale 0 - 1 (cách tính: https://en.wikipedia.org/wiki/Feature_scaling): 
def normalize_01(data):
    # Tìm LN, NN: 
    nho_nhat = data.min()
    lon_nhat = data.max(0)
    # Thực hiện chuyển hóa:
    chuan_hoa = (data - nho_nhat) / (lon_nhat - nho_nhat)
    return chuan_hoa

In [83]:
# Sử dụng hàm vừa viết để test một array: 
num_data = array([1, -2, 3, 4, 8])
normalize_01(num_data)

array([ 0.3,  0. ,  0.5,  0.6,  1. ])

### 2.3 chuẩn hóa theo mean 

In [84]:
# Viết hàm chuẩn hóa theo mean (cách tính: https://en.wikipedia.org/wiki/Feature_scaling): 
from numpy import mean, std

def chuan_hoa_mean(df): 
    return (df - df.mean()) / df.std()

In [85]:
# Test hàm vừa có: 
chuan_hoa_mean(num_data)

array([-0.54370988, -1.44989302,  0.06041221,  0.36247326,  1.57071744])

### 2.4 Ứng dụng: hồi quy chuẩn hóa (Standardized Coefficient)

In [86]:
# Load các gói và các hàm cần thiết: 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

In [87]:
# Load bộ dữ liệu cần sử dụng: 
data = sm.datasets.get_rdataset("Guerry", "HistData").data

# Và xem qua: 
data.head()

,dept,Region,Department,Crime_pers,Crime_prop,Literacy,Donations,Infants,Suicides,MainCity,...,Crime_parents,Infanticide,Donation_clergy,Lottery,Desertion,Instruction,Prostitutes,Distance,Area,Pop1831
0,1,E,Ain,28870,15890,37,5098,33120,35039,2:Med,...,71,60,69,41,55,46,13,218.372,5762,346.03
1,2,N,Aisne,26226,5521,51,8901,14572,12831,2:Med,...,4,82,36,38,82,24,327,65.945,7369,513.00
2,3,C,Allier,26747,7925,13,10973,17044,114121,2:Med,...,46,42,76,66,16,85,34,161.927,7340,298.26
3,4,E,Basses-Alpes,12935,7289,46,2733,23018,14238,1:Sm,...,70,12,37,80,32,29,2,351.399,6925,155.90
4,5,E,Hautes-Alpes,17488,8174,69,6962,23076,16171,1:Sm,...,22,23,64,79,35,7,1,320.280,5549,129.10


In [88]:
# Thực hiện hồi quy OLS với biến phụ thuộc  là Lottery, hai  biến độc lập là Literacy và Pop1831: 
hoi_quy = smf.ols("Lottery ~ Literacy + Pop1831", data = dat).fit()

# Xem kết quả:
print(hoi_quy.summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     19.95
Date:                Tue, 19 Dec 2017   Prob (F-statistic):           8.40e-08
Time:                        19:15:08   Log-Likelihood:                -381.36
No. Observations:                  86   AIC:                             768.7
Df Residuals:                      83   BIC:                             776.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     89.8137      7.683     11.690      0.0

Ở trên chúng ta thực hiện hồi quy OLS bình thường. Muốn thực hiện hồi quy chuẩn hóa chúng ta trước hết cần chuẩn hóa các biến rồi sau đó mới thực hiện hồi quy chuẩn hóa: 

In [89]:
# Sử dụng hàm chuẩn hóa đã viết ở trên:

data_small = data[["Lottery", "Literacy", "Pop1831"]]
data_chuan_hoa = data_small.apply(chuan_hoa_mean)

# Có thể kiểm tra để thấy rằng mean = 0 và sd = 1: 
data_chuan_hoa.describe()

,Lottery,Literacy,Pop1831
count,8.600000e+01,8.600000e+01,8.600000e+01
mean,-1.290957e-17,7.358455e-17,-5.912583e-16
std,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.702044e+00,-1.569669e+00,-1.677197e+00
25%,-8.510218e-01,-8.209959e-01,-6.427309e-01
50%,0.000000e+00,-7.232264e-02,-2.182036e-01
75%,8.510218e-01,7.195433e-01,4.421293e-01
max,1.702044e+00,2.000926e+00,4.108903e+00


In [90]:
# Thực hiện hồi quy chuẩn hóa: 

hoi_quy_chuan_hoa = smf.ols("Lottery ~ Literacy + Pop1831", data = data_chuan_hoa).fit()

# Xem kết quả:
print(hoi_quy_chuan_hoa.summary())


                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     19.95
Date:                Tue, 19 Dec 2017   Prob (F-statistic):           8.40e-08
Time:                        19:15:08   Log-Likelihood:                -104.64
No. Observations:                  86   AIC:                             215.3
Df Residuals:                      83   BIC:                             222.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.388e-17      0.090  -1.55e-16      1.0

In [91]:
# Đây chính là hồi quy không có hệ số chặn nên có thể thực hiện  như  sau: 
hoi_quy_chuan_hoa = smf.ols("Lottery ~ 0 + Literacy + Pop1831", data = data_chuan_hoa).fit()

# Xem kết quả:
print(hoi_quy_chuan_hoa.summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     20.19
Date:                Tue, 19 Dec 2017   Prob (F-statistic):           6.90e-08
Time:                        19:15:08   Log-Likelihood:                -104.64
No. Observations:                  86   AIC:                             213.3
Df Residuals:                      84   BIC:                             218.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Literacy      -0.3256      0.090     -3.617      0.0